In [1]:
import pandas as pd
import requests
import urllib3
import csv
import time
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

input_files = ["football.csv", "tennis.csv", "other.csv"]
output_csv = "url_status_output.csv"
output_m3u = "tung_iptv.m3u"

# --------------------------------------------------
# URL CHECK FUNCTION
# --------------------------------------------------
def check_url(url, headers=None):
    if pd.isna(url) or str(url).strip() == "":
        return "EMPTY"
    try:
        with requests.get(
            url,
            headers=headers,
            timeout=(5, 15),
            stream=True,
            allow_redirects=True,
            verify=False
        ) as response:

            if response.status_code == 200:
                chunk = next(response.iter_content(1024), None)
                if chunk:
                    return "WORKING"
                else:
                    return "NO_DATA"

            return response.status_code

    except Exception as e:
        return f"ERROR: {type(e).__name__}"


# --------------------------------------------------
# LOGO FUNCTION
# --------------------------------------------------
def channel_to_logo(channel: str) -> str:
    channel_slug = channel.lower().replace(" ", "-")
    return f"https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/logos/{channel_slug}.png"


# --------------------------------------------------
# STEP 1: Collect union of all columns
# --------------------------------------------------
manual_columns = ["7pal","a1x","kstv","supersonic","moj"]
final_columns = ["source_file", "channel"] + manual_columns

# --------------------------------------------------
# STEP 2: Open both output files
# --------------------------------------------------
with open(output_csv, "w", newline="", encoding="utf-8") as f_csv, \
     open(output_m3u, "w", encoding="utf-8") as f_m3u:

    # CSV writer
    writer = csv.DictWriter(f_csv, fieldnames=final_columns)
    writer.writeheader()
    f_csv.flush()

    # M3U header
    f_m3u.write("#EXTM3U\n")

    # --------------------------------------------------
    # Process files one by one
    # --------------------------------------------------
    for file in input_files:

        GROUP_TITLE = file.replace(".csv", "").capitalize()
        print(f"Processing {file}...")

        df = pd.read_csv(file)

        for _, row in df.iterrows():

            output_row = {col: "" for col in final_columns}
            output_row["source_file"] = file
            output_row["channel"] = row["channel"]

            channel = row["channel"]

            for col in df.columns:
                if col == "channel" or col not in manual_columns:
                    continue
                
                ext = ""
                url = str(row[col])
                status = check_url(url)
                if status != "WORKING":
                    headers = {"User-Agent": "VLC/3.0.20 LibVLC/3.0.20"}
                    status = check_url(url, headers)
                    ext = "|User-Agent=vlc/3.0.20"

                # Write status to CSV
                output_row[col] = status

                # If working → write to M3U
                if status == "WORKING":
                    f_m3u.write(
                        f'#EXTINF:-1 group-title="{GROUP_TITLE}" '
                        f'tvg-logo="{channel_to_logo(channel)}", '
                        f'{channel} ({col.upper()})\n'
                    )
                    f_m3u.write(f"{url}{ext}\n\n")
                    f_m3u.flush()

            writer.writerow(output_row)
            f_csv.flush()

            time.sleep(0.05)  # optional safety delay

print("Done.")

Processing football.csv...
Processing tennis.csv...
Processing other.csv...
Done.
